Packages

In [1]:
from pyomo.environ import *
import numpy as np
import random
import pandas as pd
from numpy import flatnonzero as find
from pypower.api import case9, ext2int, bustypes, makeBdc, rundcpf, ppoption
from pypower.idx_bus import BUS_TYPE, REF, VA, PD, LAM_P, LAM_Q, MU_VMAX, MU_VMIN
from pypower.idx_gen import PG, MU_PMAX, MU_PMIN, MU_QMAX, MU_QMIN, PMAX, PMIN, GEN_BUS
from pypower.idx_brch import PF, PT, QF, QT, RATE_A, MU_SF, MU_ST
from pypower.idx_cost import COST
import matplotlib.pyplot as plt

In [2]:
# Run PyPower case to get initial condition
ppc = case9()
ppopt = ppoption(VERBOSE=0)
pp_dcpf, success = rundcpf(ppc, ppopt)
pp_bus = pp_dcpf["bus"]
pp_branch = pp_dcpf["branch"]
pp_gen = pp_dcpf["gen"]


Converged in 0.00 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0           0.0 to 0.0
Generators         3     On-line Capacity       820.0           0.0 to 0.0
Committed Gens     3     Generation (actual)    314.0               0.0
Loads              3     Load                   315.0               0.0
  Fixed            3       Fixed                315.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           9     Losses (I^2 * Z)         0.00              0.00
Transformers       0     Branch Charging (inj)     -                0.0
Inter-ties         0     Total Inter-tie Flow     0.0               0.0
Areas     

In [3]:
## convert to internal indexing
ppc = ext2int(ppc)
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]

## get bus index lists of each type of bus
ref, pv, pq = bustypes(bus, gen)
pvpq = np.matrix(np.r_[pv, pq])

## generator info
gbus = gen[:, GEN_BUS].astype(int)    ## what buses are they at?
refgen = find(gbus == ref)

## build B matrices and phase shift injections
B, Bf, _, _ = makeBdc(baseMVA, bus, branch)
B = B.todense()
Bf = Bf.todense()

# Problem dimensions
NG = gen.shape[0] # Number of generators
NB = bus.shape[0] # Number of buses
NBr = branch.shape[0] # Number of lines
NL = 3 # Number of loads
NW = 1 # Number of wind farms

# Map generator to bus
CG = np.zeros((NB,NG))
CG[gbus,range(3)] = 1

# Map load to bus
CL = np.zeros((NB,NL))
CL[4,0] = 1
CL[6,1] = 1
CL[8,2] = 1

# Map wind farm to bus
CW = np.zeros((NB,1))
CW[5,0] = 1

# Generator capacity limit p.u.
Pmax = gen[:, PMAX]/baseMVA
Pmin = gen[:, PMIN]/baseMVA

# Line flow limit p.u.
Lmax = branch[:, RATE_A]/baseMVA
Lmin = - branch[:, RATE_A]/baseMVA

# Quadratic cost coefficients
# Convert to p.u.
CG2 = gencost[:, COST]*baseMVA**2
CG1 = gencost[:, COST+1]*baseMVA
CG0 = gencost[:, COST+2]

# Cost coefficient for wind power
CW1 = 0
CW0 = 0

# Cost coefficients for generation reserve
CRG_up = 10*np.ones((NG,1))*baseMVA
CRG_down = 10*np.ones((NG,1))*baseMVA

# Cost coefficients for load reserve
CRL_up = 9.8*np.ones((NL,1))*baseMVA
CRL_down = 9.8*np.ones((NL,1))*baseMVA

# Uncontrollable load
PL_total = np.array([90,100,125])
PL = PL_total/baseMVA

# Generator and demand set points
# Power in p.u., Va in rad
Pg0 = pp_gen[:, PG]/baseMVA
Pd0 = pp_bus[:, PD]/baseMVA
Pinj0 = (np.matmul(CG, Pg0) - Pd0)
Va0 = pp_bus[:, VA]/180*np.pi

In [4]:
# Read temperature data
weather_data = pd.read_csv("Weather_Data/NYC_Hourly_Normal_Temp.csv")
# Read wind power data
PW_data = pd.read_csv("spring_wind.csv",header=None).to_numpy()
PW_data = PW_data/500*20/baseMVA

In [5]:
Tamb = (weather_data.iloc[:,8] - 32)/1.8
Tamb_min = min(Tamb)
Tamb_10PCTL = (weather_data.iloc[:,6] - 32)/1.8
Tamb_90PCTL = (weather_data.iloc[:,7] - 32)/1.8
std_Tamb_10PCTL = (Tamb - Tamb_10PCTL)/1.282 
std_Tamb_90PCTL = (Tamb_90PCTL - Tamb)/1.282 
std_Tamb = (std_Tamb_10PCTL + std_Tamb_90PCTL)/5

In [6]:
def VS(dt,Tamb,num_TCL,Tset,delta,R,C,P_el,COP,baseMVA):
    Tdown = np.zeros(num_TCL)
    Tup = np.zeros(num_TCL)
    T_ON = np.zeros(num_TCL)
    T_OFF = np.zeros(num_TCL)
    D = np.zeros(num_TCL)
    B = 0
    Pdown = 0
    Pup = 0
    Sdown = 0
    Sup = 0
    for i in range(0,num_TCL):
        Tdown[i] = Tset[i] - delta/2
        Tup[i] = Tset[i] + delta/2
        T_ON[i] = C[i]*R[i]*np.log((Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])/(Tup[i] - Tamb - R[i]*P_el[i]*COP[i]))
        if ((Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])/(Tup[i] - Tamb - R[i]*P_el[i]*COP[i]) < 0):
            print(Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])
            print(Tup[i] - Tamb - R[i]*P_el[i]*COP[i])
        T_OFF[i] = C[i]*R[i]*np.log((Tup[i] - Tamb)/(Tdown[i] - Tamb))
        D[i] = T_ON[i]/(T_ON[i] + T_OFF[i])
        if ( Tamb < Tdown[i] and Tamb + R[i]*P_el[i]*COP[i] > Tup[i] ):
            B = B + P_el[i]*D[i]
            Pup = Pup + P_el[i]
            Sup = Sup + dt*P_el[i]*(1 - D[i])
    return [B/1000/baseMVA,Pdown/1000/baseMVA,Pup/1000/baseMVA,Sdown,Sup]

In [7]:
# Optimization: Problem formulation
def optimizer(NG,NL,Tamb_sample,PW_sample,num_TCL,Tset,delta,R,C,P_el,COP,S0,PL,\
    Pmax,Pmin,Lmax,Lmin,CG,CW,CL,CG0,CG1,CG2,CW0,CW1,CRG_up,CRG_down,CRL_up,CRL_down,nsample,PW_f):
    model = ConcreteModel()
    # Defining the variables
    model.PG = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.PC = Var(range(NL),within = NonNegativeReals, initialize=0)
    model.RG_down = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.RG_up = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.RG = Var(range(NG),range(nsample),within = Reals, initialize=0)
    model.RL_down = Var(range(NL),within = NonNegativeReals, initialize=0)
    model.RL_up = Var(range(NL),within = NonNegativeReals, initialize=0)
    model.RL = Var(range(NL),range(nsample),within = Reals, initialize=0)
    model.dG_down = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.dG_up = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.dL_down = Var(range(NL),within = NonNegativeReals, initialize=0)
    model.dL_up = Var(range(NL),within = NonNegativeReals, initialize=0)
    model.Va = Var(range(NB), within=Reals, initialize=0, bounds=(-1,1))
    model.s1 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s2 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s3 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s4 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s5 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s6 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.d = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.e = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.dual = Suffix(direction=Suffix.IMPORT)

    ## Defining the deterministic constraints
    model.c = ConstraintList()

    # Power mismatch distribution
    model.c.add(sum(model.dG_down[i] for i in range(NG)) + sum(model.dL_up[j] for j in range(NL)) == 1)
    model.c.add(sum(model.dG_up[i] for i in range(NG)) + sum(model.dL_down[j] for j in range(NL)) == 1)
    
    # Line flow limit constraints
    for i in range(NBr):
        model.c.add(sum(Bf[i,j]*model.Va[j] for j in range(9)) <= Lmax[i])
        model.c.add(sum(Bf[i,j]*model.Va[j] for j in range(9)) >= Lmin[i])
    
    PB_all = np.zeros((NL,nsample))
    P_mis_wind_all = np.zeros(nsample)
    identified_VS_all = np.zeros((NL,nsample,5))
    
    # Defining the CC constraints
    for i in range(nsample):

        for j in range(NG):
        # Generation capacity limit
            model.c.add(model.PG[j] + model.RG[j,i] >= Pmin[j])
            model.c.add(model.PG[j] + model.RG[j,i] <= Pmax[j])
            # Reserve capacity limit
            model.c.add(model.RG[j,i] >= -model.RG_down[j])
            model.c.add(model.RG[j,i] <= model.RG_up[j])
    
        # DC Power flow constraint
        for j in range(NB):
            model.c.add(sum(CG[j,k]*model.PG[k] + CG[j,k]*model.RG[k,i] for k in range(NG)) + CW[j]*PW_sample[i]\
            - sum(CL[j,k]*PL[k] + CL[j,k]*model.PC[k] + CL[j,k]*model.RL[k,i] for k in range(NL)) >= sum(B[j,k]*model.Va[k] for k in range(NB)) - model.s1[i])
            model.c.add(sum(CG[j,k]*model.PG[k] + CG[j,k]*model.RG[k,i] for k in range(NG)) + CW[j]*PW_sample[i]\
            - sum(CL[j,k]*PL[k] + CL[j,k]*model.PC[k] + CL[j,k]*model.RL[k,i] for k in range(NL)) <= sum(B[j,k]*model.Va[k] for k in range(NB)) + model.s2[i])
            # model.c.add(sum(CG[j,k]*model.PG[k] + CG[j,k]*model.RG[k,i] for k in range(NG)) + CW[j]*PW_sample[i]\
            # - sum(CL[j,k]*PL[k] + CL[j,k]*model.PC[k] for k in range(NL)) >= sum(B[j,k]*model.Va[k] for k in range(NB)) - model.s1[i])
            # model.c.add(sum(CG[j,k]*model.PG[k] + CG[j,k]*model.RG[k,i] for k in range(NG)) + CW[j]*PW_sample[i]\
            # - sum(CL[j,k]*PL[k] + CL[j,k]*model.PC[k] for k in range(NL)) <= sum(B[j,k]*model.Va[k] for k in range(NB)) + model.s2[i])
        
        P_mis_wind = PW_sample[i] - PW_f
        P_mis_wind_all[i] = P_mis_wind

        PB = np.zeros(NL)

        # print(i)

        # P_mis_load = 0
        for j in range(NL):
            identified_VS = VS(0.25,Tamb_sample[i],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
            # print("VS0", j, identified_VS[0])
            identified_VS_all[j,i,:] = identified_VS
            PB[j] = identified_VS[0]
        
        PB_all[:,i] = PB
        
        model.c.add(model.e[i] >= P_mis_wind - sum(model.PC[j] for j in range(NL)) + sum(PB[j] for j in range(NL)))
        model.c.add(model.d[i] >= - P_mis_wind + sum(model.PC[j] for j in range(NL)) - sum(PB[j] for j in range(NL)))
        
        for j in range(NG):
            model.c.add(model.RG[j,i] == model.dG_up[j]*model.d[i] - model.dG_down[j]*model.e[i])
        

        for j in range(NL):
            model.c.add(model.RL[j,i] == model.dL_up[j]*model.e[i] - model.dL_down[j]*model.d[i])
            identified_VS = VS(0.25,Tamb_sample[i],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
            model.c.add(model.PC[j] + model.RL[j,i] >= identified_VS[1])
            model.c.add(model.PC[j] + model.RL[j,i] <= identified_VS[2])
            model.c.add(model.PC[j] >= identified_VS[1])
            model.c.add(model.PC[j] <= identified_VS[2])
            model.c.add(model.RL[j,i] >= -model.RL_down[j])
            model.c.add(model.RL[j,i] <= model.RL_up[j])
            model.c.add(S0[j] + 0.25*(model.PC[j] + model.RL[j,i] - identified_VS[0])*1000*baseMVA >= identified_VS[3] - model.s3[i])
            model.c.add(S0[j] + 0.25*(model.PC[j] + model.RL[j,i] - identified_VS[0])*1000*baseMVA  <= identified_VS[4] + model.s4[i])
            model.c.add(S0[j] + 0.75*(model.PC[j] - identified_VS[0])*1000*baseMVA  + 0.25*(model.PC[j] + model.RL[j,i]\
                 - identified_VS[0])*1000*baseMVA  >= identified_VS[3] - model.s5[i])
            model.c.add(S0[j] + 0.75*(model.PC[j] - identified_VS[0])*1000*baseMVA  + 0.25*(model.PC[j] + model.RL[j,i]\
                 - identified_VS[0])*1000*baseMVA  <= identified_VS[4] + model.s6[i])        
    
    def cost(model, CG0, CG1, CG2, CRG_down, CRG_up, nsample, CRL_down, CRL_up):

        cost = sum(CG0[i] for i in range(NG))\
                + sum(CG1[i]*model.PG[i] for i in range(NG))\
                + sum(CG2[i]*model.PG[i]**2 for i in range(NG))\
                + sum(CRG_up[i]*model.RG_up[i] for i in range(NG))\
                + sum(CRG_down[i]*model.RG_down[i] for i in range(NG))\
                + 1000*sum(model.s1[i] + model.s2[i] for i in range(nsample))\
                + 1e10*sum(model.s3[i] + model.s4[i] + model.s5[i] + model.s6[i] for i in range(nsample))\
                + 1000*sum(model.d[i] + model.e[i] for i in range(nsample))\
                + sum(CRL_up[i]*model.RL_up[i] for i in range(NL))\
                + sum(CRL_down[i]*model.RL_down[i] for i in range(NL))
                
                
                
        return cost

    model.obj = Objective(expr=cost(model, CG0, CG1, CG2, CRG_down, CRG_up, nsample, CRL_down, CRL_up), sense=minimize)

    
    return model, PB_all, P_mis_wind, identified_VS_all

In [8]:
# Parameters related to the demand side
num_TCL = 5000
Tset = np.zeros((num_TCL,NL))
R = np.zeros((num_TCL,NL))
C = np.zeros((num_TCL,NL))
COP = np.zeros((num_TCL,NL))
P_el = np.zeros((num_TCL,NL))
Tset_set = [21,21.5,22,22.5,23]
for i in range(num_TCL): 
    for j in range(NL):
        s = random.sample(Tset_set,1)
        Tset[i,j] = s[0] 
delta = 1
R_nom = 2
R_low = 0.8*R_nom
R_high = 1.2*R_nom
C_nom = 2
C_low = 0.8*C_nom
C_high = 1.2*C_nom
COP_nom = 2.5
COP_low = 0.8*COP_nom
COP_high = 1.2*COP_nom
for i in range(NL): 
    R[:,i] = np.random.uniform(R_low,R_high,num_TCL)
    C[:,i] = np.random.uniform(C_low,C_high,num_TCL)
    COP[:,i] = np.random.uniform(COP_low,COP_high,num_TCL)
    for j in range(num_TCL):
        P_el[j,i] = np.ceil((Tset[j,i] + delta/2 - Tamb_min)/(R[j,i]*COP[j,i])) + 3
# Sampling
eps = 0.1
beta = 0.1
num_decision_variables = 5*NG + 5*NL
num_samples = np.ceil((2/eps)*(np.log(1/beta) + num_decision_variables)).astype(int)

# np.random.seed(1)
# Tamb_sample = np.random.normal(mu_Tamb,sigma_Tamb,num_samples)
# PW_sample = np.random.choice(PW_data[i,:], size=num_samples)

P_B = np.zeros((NL,1))
Pdown = np.zeros((NL,1))
Pup = np.zeros((NL,1))
Sdown = np.zeros((NL,1))
Sup = np.zeros((NL,1))
S0 = np.zeros((NL,1))

for j in range(NL):
    [P_B[j],Pdown[j],Pup[j],Sdown[j],Sup[j]] = \
        VS(0.25,Tamb[0],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
    S0[j] = (Sdown[j] + Sup[j])/2    


In [25]:
# Define the model
nsample= 50
np.random.seed(1)
Tamb_sample = np.random.normal(Tamb[0],std_Tamb[0],num_samples)
PW_sample = np.random.choice(PW_data[0,:], size=num_samples)
PW_f = np.median(PW_data[0,:])
# PW_f = np.median(PW_data)
model, PB_all, P_mis_wind, identified_VS_all = optimizer(NG,NL,
Tamb_sample,PW_sample,num_TCL,Tset,delta,R,C,P_el,COP,S0,PL,\
    Pmax,Pmin,Lmax,Lmin,CG,CW,CL,CG0,CG1,CG2,CW0,CW1,CRG_up,CRG_down,CRL_up,CRL_down,nsample,PW_f)

# Solve the problem
solver = SolverFactory("bonmin")
solver.solve(model)

model.display()

Model unknown

  Variables:
    PG : Size=3, Index=PG_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 0.9921561973918664 :  None : False : False : NonNegativeReals
          1 :     0 : 1.5074956128375114 :  None : False : False : NonNegativeReals
          2 :     0 :  1.054181047621389 :  None : False : False : NonNegativeReals
    PC : Size=3, Index=PC_index
        Key : Lower : Value               : Upper : Fixed : Stale : Domain
          0 :     0 : 0.21949256016566993 :  None : False : False : NonNegativeReals
          1 :     0 : 0.22006860876190926 :  None : False : False : NonNegativeReals
          2 :     0 : 0.21855556802429263 :  None : False : False : NonNegativeReals
    RG_down : Size=3, Index=RG_down_index
        Key : Lower : Value                : Upper : Fixed : Stale : Domain
          0 :     0 :  0.00689678768862343 :  None : False : False : NonNegativeReals
          1 :     0 : 0.006896787706885536 :  No

In [9]:
def cost(model,CG0,CG1,CG2,CRG_up,CRG_down,CRL_up,CRL_down,NG,NL):
    r_pg = np.zeros(NG)
    r_rg_up = np.zeros(NG)
    r_rg_down = np.zeros(NG)
    r_rl_up = np.zeros(NL)
    r_rl_down = np.zeros(NL)
    r_pc = np.zeros(NL)
    gen_cost = np.zeros(NG)
    gen_r_cost = np.zeros(NG)
    load_r_cost = np.zeros(NL)


    for i in range(NG):
        r_pg[i] = model.PG[i]()
        r_rg_up[i] = model.RG_up[i]()
        r_rg_down[i] = model.RG_down[i]()
        gen_cost[i] = CG0[i] + CG1[i]*r_pg[i] + CG2[i]*r_pg[i]**2
        gen_r_cost[i] = CRG_up[i]*r_rg_up[i] + CRG_down[i]*r_rg_down[i]

    for i in range(NL):
        r_rl_up[i] = model.RL_up[i]()
        r_rl_down[i] = model.RL_down[i]()
        r_pc[i] = model.PC[i]()
        load_r_cost[i] = CRL_up[i]*r_rl_up[i] + CRL_down[i]*r_rl_down[i]

    total_cost = np.sum(gen_cost + gen_r_cost) + np.sum(load_r_cost)

    return r_pg,r_rg_up,r_rg_down,r_rl_up,r_rl_down,r_pc,gen_cost,gen_r_cost,load_r_cost,total_cost

In [25]:
r_pg,r_rg_up,r_rg_down,r_rl_up,r_rl_down,r_pc,gen_cost,gen_r_cost,load_r_cost,total_cost = \
    cost(model,CG0,CG1,CG2,CRG_up,CRG_down,CRL_up,CRL_down,NG,NL)

In [26]:
r_pc

array([0.21865266, 0.218126  , 0.218362  ])

In [38]:
S0_all = np.zeros((24,NL))
R_PC = np.zeros((24,NL))
models = []

# Initialize at hour 0
for j in range(NL):
    [P_B[j],Pdown[j],Pup[j],Sdown[j],Sup[j]] = \
        VS(0.25,Tamb[0],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
    S0_all[0,j] = (Sdown[j] + Sup[j])/2  
    # S0[0,j] = 0

nsample= 50
h = 0
for h in range(24):
    np.random.seed(1)
    Tamb_sample = np.random.normal(Tamb[h],std_Tamb[h],num_samples)
    PW_sample = np.random.choice(PW_data[h,:], size=num_samples)
    PW_f = np.median(PW_data[h, :])

    # Define the problem
    model, PB_all, P_mis_wind, identified_VS_all = optimizer(NG,NL,Tamb_sample,PW_sample,num_TCL,Tset,delta,R,C,P_el,COP,S0_all[h,:],PL,\
        Pmax,Pmin,Lmax,Lmin,CG,CW,CL,CG0,CG1,CG2,CW0,CW1,CRG_up,CRG_down,CRL_up,CRL_down,nsample,PW_f)

    # Solve the problem
    solver = SolverFactory("bonmin")
    solver.solve(model)
    r_pg,r_rg_up,r_rg_down,r_rl_up,r_rl_down,r_pc,gen_cost,gen_r_cost,load_r_cost,total_cost = \
        cost(model,CG0,CG1,CG2,CRG_up,CRG_down,CRL_up,CRL_down,NG,NL)
    R_PC[h,:] = r_pc

    models.append(model)

    if h != 23:
        for j in range(NL):
            [P_B[j],Pdown[j],Pup[j],Sdown[j],Sup[j]] = \
                VS(0.25,Tamb[h],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
            S0_all[h+1,j] = S0_all[h,j] + r_pc[j]*1000*baseMVA - P_B[j]*1000*baseMVA
    


In [39]:
print("S0: \n", S0_all)
print("PC: \n", R_PC)
print(P_B)

S0: 
 [[2794.52310205 2798.6219178  2795.10051574]
 [3543.96393504 3554.45813643 3531.46144406]
 [3575.1206531  3584.14952151 3561.11930727]
 [3503.23249585 3506.16628399 3510.93075784]
 [3519.47330194 3522.44483179 3526.08337709]
 [3437.01979884 3439.77971456 3444.96308986]
 [3475.63240536 3478.49001855 3483.45395921]
 [3497.88112887 3500.75489824 3505.65466735]
 [3528.59143194 3531.27943549 3536.24709037]
 [3522.55976602 3524.99557008 3530.23397666]
 [3749.30675108 3752.09623049 3756.26780581]
 [3823.6431119  3823.29243293 3833.74357492]
 [3946.31871241 3946.94130269 3953.77002691]
 [4133.89727029 4138.41362907 4137.60537665]
 [4192.17928064 4196.06781236 4195.86399433]
 [4147.87322988 4152.42633798 4153.57315364]
 [4137.94262386 4142.89187956 4143.1077364 ]
 [4121.76195903 4126.80325873 4126.84998691]
 [4059.10038898 4063.8096669  4064.52128857]
 [4110.38226247 4114.93141222 4116.07878549]
 [4015.01052252 4019.1373521  4021.13294779]
 [3899.56152258 3903.12641223 3906.0486542 ]
 [37

In [1870]:
identified_VS = VS(0.25,-10,num_TCL,Tset[:,0],delta,R[:,0],C[:,0],P_el[:,0],COP[:,0],baseMVA)
identified_VS

[0.06557789654450581, 0.0, 0.08681, 0, 530.8025863873563]

Model unknown

  Variables:
    PG : Size=3, Index=PG_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 0.8668609079126465 :  None : False : False : NonNegativeReals
          1 :     0 : 1.3453363411848764 :  None : False : False : NonNegativeReals
          2 :     0 : 0.9416685513699042 :  None : False : False : NonNegativeReals
    PC : Size=3, Index=PC_index
        Key : Lower : Value               : Upper : Fixed : Stale : Domain
          0 :     0 : 0.04651441041440668 :  None : False : False : NonNegativeReals
          1 :     0 :  0.0469968134383421 :  None : False : False : NonNegativeReals
          2 :     0 : 0.04638701288333312 :  None : False : False : NonNegativeReals
    RG_down : Size=3, Index=RG_down_index
        Key : Lower : Value               : Upper : Fixed : Stale : Domain
          0 :     0 : 0.05780660536685922 :  None : False : False : NonNegativeReals
          1 :     0 : 0.05780660907041171 :  None 

In [1872]:
model.RG_up.pprint()
model.RG_down.pprint()
model.RL_up.pprint()
model.RL_down.pprint()

RG_up : Size=3, Index=RG_up_index
    Key : Lower : Value                  : Upper : Fixed : Stale : Domain
      0 :     0 :  1.379558389311056e-06 :  None : False : False : NonNegativeReals
      1 :     0 : 1.3795551434055288e-06 :  None : False : False : NonNegativeReals
      2 :     0 : 1.3795583325291806e-06 :  None : False : False : NonNegativeReals
RG_down : Size=3, Index=RG_down_index
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :     0 : 0.05780660536685922 :  None : False : False : NonNegativeReals
      1 :     0 : 0.05780660907041171 :  None : False : False : NonNegativeReals
      2 :     0 :  0.0578066044601672 :  None : False : False : NonNegativeReals
RL_up : Size=3, Index=RL_up_index
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
      0 :     0 : 0.0007051821532996659 :  None : False : False : NonNegativeReals
      1 :     0 : 1.976687702077583e-06 :  None : False : False : NonNegativeReals
      2 :     0

In [1873]:
max_d = 0
max_e = 0
for i in range(nsample):
    if max_d <= model.d[i]():
        max_d = model.d[i]()
    if max_e <= model.e[i]():
        max_e = model.e[i]()

print(max_d)
print(max_e)

0.0659238310219688
0.1749672158578191


In [1874]:
model.s3.pprint()
model.s4.pprint()
model.s5.pprint()
model.s6.pprint()

s3 : Size=50, Index=s3_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :   0.0 :  None : False : False : NonNegativeReals
      1 :     0 :   0.0 :  None : False : False : NonNegativeReals
      2 :     0 :   0.0 :  None : False : False : NonNegativeReals
      3 :     0 :   0.0 :  None : False : False : NonNegativeReals
      4 :     0 :   0.0 :  None : False : False : NonNegativeReals
      5 :     0 :   0.0 :  None : False : False : NonNegativeReals
      6 :     0 :   0.0 :  None : False : False : NonNegativeReals
      7 :     0 :   0.0 :  None : False : False : NonNegativeReals
      8 :     0 :   0.0 :  None : False : False : NonNegativeReals
      9 :     0 :   0.0 :  None : False : False : NonNegativeReals
     10 :     0 :   0.0 :  None : False : False : NonNegativeReals
     11 :     0 :   0.0 :  None : False : False : NonNegativeReals
     12 :     0 :   0.0 :  None : False : False : NonNegativeReals
     13 :     0 :   0.0 :  None : False : F

In [1875]:
total_RG_down = 0
total_RG_up = 0

for i in range(NG):
    total_RG_down += model.RG_down[i]()
    total_RG_up += model.RG_up[i]()

print(total_RG_down)
print(total_RG_up)

# total_RL_down = 0
# total_RL_up = 0

# for i in range(NL):
#     total_RL_down += model.RL_down[i]()
#     total_RL_up += model.RL_up[i]()

# print(total_RL_down)
# print(total_RL_up)

0.17341981889743813
4.1386718652457655e-06


In [1876]:
model.PC.pprint()

PC : Size=3, Index=PC_index
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :     0 : 0.04651441041440668 :  None : False : False : NonNegativeReals
      1 :     0 :  0.0469968134383421 :  None : False : False : NonNegativeReals
      2 :     0 : 0.04638701288333312 :  None : False : False : NonNegativeReals


In [1889]:
print("Pmax: ", Pmax)
print("Pg: ", r_pg)
print("RG_up: ", r_rg_up)
print("RG_down: ", r_rg_down)
print("RL_up: ", r_rl_up)
print("RL_down: ", r_rl_down)

Pmax:  [2.5 3.  2.7]
Pg:  [[0.86686091]
 [1.34533634]
 [0.94166855]]
RG_up:  [[1.37955839e-06]
 [1.37955514e-06]
 [1.37955833e-06]]
RG_down:  [[0.05780661]
 [0.05780661]
 [0.0578066 ]]
RL_up:  [[7.05182153e-04]
 [1.97668770e-06]
 [8.40488065e-04]]
RL_down:  [[0.02179309]
 [0.02251172]
 [0.02161913]]


In [1884]:
print(total_cost)
print(gen_cost)

5464.8728680346385
[[1410.02307099]
 [2299.88075122]
 [1515.42293942]]
